# EECS 3401 Project 

## Filght Status Prediction


#### Can we predict if the flight is diverted?

In [216]:
import zipfile
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [217]:
# Specify the path to your compressed folder and the CSV file inside
compressed_folder_path = 'Combined_Flights_2022.zip'
csv_file_name = 'Combined_Flights_2022.csv'

# Extract the contents of the ZIP file
with zipfile.ZipFile(compressed_folder_path, 'r') as zip_ref:
    zip_ref.extractall('path/to/extracted/folder')

# Create the full path to the extracted CSV file
csv_file_path = f'path/to/extracted/folder/{csv_file_name}'

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path)
#df = pd.read_csv('Combined_Flights_2022.csv')
df.head(10)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
6,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,LCH,False,False,2139,2136.0,0.0,-3.0,...,2147.0,2213.0,5.0,2231,-13.0,0.0,-1.0,2200-2259,1,0
7,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",TYS,IAH,False,False,1129,1117.0,0.0,-12.0,...,1139.0,1255.0,16.0,1306,5.0,0.0,0.0,1300-1359,4,0
8,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,AEX,False,False,1424,1414.0,0.0,-10.0,...,1430.0,1507.0,6.0,1524,-11.0,0.0,-1.0,1500-1559,1,0
9,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,MOB,False,False,954,947.0,0.0,-7.0,...,1004.0,1104.0,6.0,1121,-11.0,0.0,-1.0,1100-1159,2,0


In [218]:
columns = df.columns
len(columns)

61

In [219]:
columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [220]:
len(df.index)

4078318

In [221]:
origin_airtport_city = df['Origin'].unique()
origin_airtport_city

array(['GJT', 'HRL', 'DRO', 'IAH', 'DEN', 'TYS', 'MOB', 'LNK', 'BHM',
       'IAD', 'ECP', 'AEX', 'LFT', 'BTR', 'ABQ', 'BUF', 'ALB', 'HSV',
       'TUL', 'CPR', 'CRP', 'EWR', 'DCA', 'STL', 'CVG', 'RIC', 'PVD',
       'CLT', 'GSO', 'ERI', 'ORD', 'LEX', 'ABE', 'CAK', 'AVL', 'DAY',
       'CHO', 'ROA', 'PWM', 'LIT', 'MKE', 'BTV', 'OMA', 'GRB', 'MLI',
       'SDF', 'FNT', 'CHS', 'PIT', 'TVC', 'ATW', 'DLH', 'ROC', 'SPI',
       'HPN', 'MBS', 'CID', 'MYR', 'CAE', 'SGF', 'AVP', 'GRR', 'DSM',
       'CLE', 'PIA', 'MSN', 'MDT', 'SBN', 'MEM', 'ICT', 'SHV', 'XNA',
       'DIK', 'GPT', 'PNS', 'LRD', 'LCH', 'MFE', 'FLG', 'BRO', 'SAF',
       'PQI', 'ORF', 'LGA', 'SYR', 'RDU', 'ILM', 'IND', 'PHL', 'CMH',
       'SCE', 'ITH', 'JAX', 'OKC', 'JAN', 'GSP', 'TXK', 'ELP', 'MAF',
       'AUS', 'IDA', 'BNA', 'SAV', 'AMA', 'ATL', 'BDL', 'BLI', 'BOI',
       'BOS', 'BUR', 'BWI', 'BZN', 'COS', 'DAL', 'DTW', 'EUG', 'FAT',
       'FLL', 'GEG', 'HDN', 'HNL', 'HOU', 'ISP', 'ITO', 'KOA', 'LAS',
       'LAX', 'LBB',

In [222]:
busiest_airports = df['Origin'].value_counts().nlargest(10)
busiest_airports

Origin
ATL    183697
ORD    172169
DFW    161218
DEN    158092
CLT    126088
LAX    111935
LGA    100260
SEA     98884
LAS     97674
PHX     96095
Name: count, dtype: int64

In [223]:
specific_origin_airport = ['ATL','ORD','DFW','DEN','CLT']  # Replace with the airport code you're interested in
df_filtered = df[df['Origin'].isin(specific_origin_airport)]
df_filtered.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
10,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,CPR,False,False,1540,1546.0,6.0,6.0,...,1559.0,1644.0,6.0,1650,0.0,0.0,0.0,1600-1659,1,0
15,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,GJT,False,False,941,937.0,0.0,-4.0,...,949.0,1028.0,3.0,1049,-18.0,0.0,-2.0,1000-1059,1,0
16,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,DRO,False,False,1330,1322.0,0.0,-8.0,...,1340.0,1425.0,2.0,1444,-17.0,0.0,-2.0,1400-1459,2,0
17,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,SAF,False,False,1122,1112.0,0.0,-10.0,...,1124.0,1209.0,4.0,1241,-28.0,0.0,-2.0,1200-1259,2,0


In [224]:
# Create a new column 'Delay' based on the condition
df_filtered['Delay'] = (df_filtered['DepDelayMinutes'] > 0).astype(int)
df_filtered[['Delay','DepDelayMinutes']]

C:\Users\aumpa\AppData\Local\Temp\ipykernel_16504\3319011012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Delay'] = (df_filtered['DepDelayMinutes'] > 0).astype(int)


,Delay,DepDelayMinutes
5,0,0.0
10,1,6.0
15,0,0.0
16,0,0.0
17,0,0.0
...,...,...
4078284,1,2.0
4078288,1,69.0
4078304,0,0.0
4078305,1,54.0


## Dropping Unwanted columns

These columns represent unique IDs and code which don't provide much value
in the training the model and are unrealated to the data analysis.

In [225]:
# List of columns to drop (IDs)
id_columns_to_drop = ['DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'OriginAirportID',
                      'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID',
                      'DestCityMarketID', 'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline',
                      'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number',
                      'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
                   'OriginCityName', 'OriginState', 'DestCityName', 'DestState',
                      'OriginStateName','OriginStateFips','OriginWac'
                  ,'DestStateName','DestStateFips','DestWac','Operating_Airline']

# Drop ID columns from the DataFrame
df_filtered = df_filtered.drop(columns=id_columns_to_drop)

# Print data types of the remaining columns
print(df_filtered.dtypes)

FlightDate               object
Airline                  object
Origin                   object
Dest                     object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
DepDelay                float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Year                      int64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDelay                float64
ArrDel15                float64
ArrivalD

## 4. Prepare the data for Machine Learning algorithms.

### - Exploring the columns.

In [226]:
columns = df_filtered.columns
len(columns)

36

In [227]:
len(df_filtered.index)

801264

In [228]:
airlines = df_filtered['Airline'].unique()
len(airlines)

20

In [229]:
dest = df_filtered['Dest'].unique()
len(dest)

300

In [230]:
df_filtered[['DepDelay','DepDelayMinutes','ArrTimeBlk','DepTimeBlk','FlightDate','Origin','Dest','DepartureDelayGroups']]

,DepDelay,DepDelayMinutes,ArrTimeBlk,DepTimeBlk,FlightDate,Origin,Dest,DepartureDelayGroups
5,-3.0,0.0,1200-1259,0900-0959,2022-04-04,DEN,TUL,-1.0
10,6.0,6.0,1600-1659,1500-1559,2022-04-04,DEN,CPR,0.0
15,-4.0,0.0,1000-1059,0900-0959,2022-04-04,DEN,GJT,-1.0
16,-8.0,0.0,1400-1459,1300-1359,2022-04-04,DEN,DRO,-1.0
17,-10.0,0.0,1200-1259,1100-1159,2022-04-04,DEN,SAF,-1.0
...,...,...,...,...,...,...,...,...
4078284,2.0,2.0,1600-1659,1300-1359,2022-03-23,ORD,CLT,0.0
4078288,69.0,69.0,1400-1459,1000-1059,2022-03-24,ORD,JAX,4.0
4078304,-4.0,0.0,1400-1459,1200-1259,2022-03-31,ATL,EWR,-1.0
4078305,54.0,54.0,1300-1359,1100-1159,2022-03-31,CLT,EWR,3.0


We can observe follwing - 
1. There are 20 unique airlines taking off from the Atlante airport. This information is required in model training.
2. There are 300 destination.
3. ArrTimeBlk and DepTimeBlk columns are represent hourly intervals so we will take in consideration the start of the hour
for example - '1000-1059' we will on consider '10' hours.
4. The columns representing _Delay and _DelayMinutes represent same things but have slight differnce in handling early departures and early arrivals
for our model we to check by using both columns separtly and compare which contributes to more accurate models.
5. Orgin column will be dropped cause its redundant for us.
6. We will also drop the year and quarter columns, the  main reason for that is we are taking data from year 2022 thats why its totally 
useless for us.
7. We need to convert the columns reperesenting time in 'hhmm' format using sine and cosine function to capture their cyclical nature,
if we don't do so then the ml algorithms will plot time as continous form which is not the case


### - Dropping the uneccesary columns

In [231]:
# dropping columns that are not useful for model training.
# Initially we will drop the ArrDelay and DepDelay for the first run
list_cols_drop_ml = ['FlightDate','Year','Dest', 'DepDelay','ArrDelay','Quarter']
df_filtered = df_filtered.drop(columns=list_cols_drop_ml)
print(df_filtered.dtypes)



Airline                  object
Origin                   object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDel15                float64
ArrivalDelayGroups      float64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
Delay                     int32
dtype: object


In [232]:
columns = df_filtered.columns
print('Total columns - ' + str(len(columns)))

Total columns - 30


### - Extracting hour from the ArrTimeBlk and DepTimeBlk time intervals

In [233]:
# 'ArrTimeBlk' and 'DepTimeBlk' are the columns with time block intervals
df_filtered['ArrTimeBlk_numeric'] = df_filtered['ArrTimeBlk'].str.split('-').str[0].astype(int)

# Repeat the process for 'DepTimeBlk' 
df_filtered['DepTimeBlk_numeric'] = df_filtered['DepTimeBlk'].str.split('-').str[0].astype(int)

# Print data types of the updated DataFrame
print(df_filtered.dtypes)

Airline                  object
Origin                   object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDel15                float64
ArrivalDelayGroups      float64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
Delay                     int32
ArrTimeBlk_numeric        int32
DepTimeB

In [234]:
# dropping the 'ArrTimeBlk' and DepTimeBlk
df_filtered = df_filtered.drop(columns=['DepTimeBlk', 'ArrTimeBlk'])
df_filtered.head(5)

,Airline,Origin,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,ArrTime,ArrDelayMinutes,AirTime,...,WheelsOn,TaxiIn,CRSArrTime,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings,Delay,ArrTimeBlk_numeric,DepTimeBlk_numeric
5,"Commutair Aka Champlain Enterprises, Inc.",DEN,False,False,955,952.0,0.0,1238.0,0.0,77.0,...,1234.0,4.0,1240,0.0,-1.0,3,0,0,1200,900
10,"Commutair Aka Champlain Enterprises, Inc.",DEN,False,False,1540,1546.0,6.0,1650.0,0.0,45.0,...,1644.0,6.0,1650,0.0,0.0,1,0,1,1600,1500
15,"Commutair Aka Champlain Enterprises, Inc.",DEN,False,False,941,937.0,0.0,1031.0,0.0,39.0,...,1028.0,3.0,1049,0.0,-2.0,1,0,0,1000,900
16,"Commutair Aka Champlain Enterprises, Inc.",DEN,False,False,1330,1322.0,0.0,1427.0,0.0,45.0,...,1425.0,2.0,1444,0.0,-2.0,2,0,0,1400,1300
17,"Commutair Aka Champlain Enterprises, Inc.",DEN,False,False,1122,1112.0,0.0,1213.0,0.0,45.0,...,1209.0,4.0,1241,0.0,-2.0,2,0,0,1200,1100


### - Handling columns with value in 'hhmm' format

In [235]:
# Columns that are in the "hhmm" format
df_filtered[['CRSDepTime','DepTime','ArrTime','CRSArrTime','WheelsOff','WheelsOn','ArrTimeBlk_numeric','DepTimeBlk_numeric']]


,CRSDepTime,DepTime,ArrTime,CRSArrTime,WheelsOff,WheelsOn,ArrTimeBlk_numeric,DepTimeBlk_numeric
5,955,952.0,1238.0,1240,1017.0,1234.0,1200,900
10,1540,1546.0,1650.0,1650,1559.0,1644.0,1600,1500
15,941,937.0,1031.0,1049,949.0,1028.0,1000,900
16,1330,1322.0,1427.0,1444,1340.0,1425.0,1400,1300
17,1122,1112.0,1213.0,1241,1124.0,1209.0,1200,1100
...,...,...,...,...,...,...,...,...
4078284,1329,1331.0,1656.0,1635,1408.0,1645.0,1600,1300
4078288,1046,1155.0,1528.0,1414,1207.0,1523.0,1400,1000
4078304,1202,1158.0,1404.0,1422,1216.0,1358.0,1400,1200
4078305,1134,1228.0,1413.0,1335,1249.0,1408.0,1300,1100


In [236]:
time_columns = ['CRSDepTime','DepTime','ArrTime','CRSArrTime','WheelsOff','WheelsOn','ArrTimeBlk_numeric','DepTimeBlk_numeric']

for column in time_columns:
    # Extract hour and minute
    df_filtered[column + '_hour'] = df_filtered[column] // 100
    df_filtered[column + '_minute'] = df_filtered[column] % 100

    # Apply trigonometric transformations
    df_filtered[column + '_sin'] = np.sin(2 * np.pi * (df_filtered[column + '_hour'] * 60 + df_filtered[column + '_minute']) / (24 * 60))
    df_filtered[column + '_cos'] = np.cos(2 * np.pi * (df_filtered[column + '_hour'] * 60 + df_filtered[column + '_minute']) / (24 * 60))

    # Drop the original columns
    df_filtered = df_filtered.drop(columns=[column, column + '_hour', column + '_minute'])

# Print data types of the updated DataFrame
print(df_filtered.dtypes)

Airline                    object
Origin                     object
Cancelled                    bool
Diverted                     bool
DepDelayMinutes           float64
ArrDelayMinutes           float64
AirTime                   float64
CRSElapsedTime            float64
ActualElapsedTime         float64
Distance                  float64
Month                       int64
DayofMonth                  int64
DayOfWeek                   int64
DepDel15                  float64
DepartureDelayGroups      float64
TaxiOut                   float64
TaxiIn                    float64
ArrDel15                  float64
ArrivalDelayGroups        float64
DistanceGroup               int64
DivAirportLandings          int64
Delay                       int32
CRSDepTime_sin            float64
CRSDepTime_cos            float64
DepTime_sin               float64
DepTime_cos               float64
ArrTime_sin               float64
ArrTime_cos               float64
CRSArrTime_sin            float64
CRSArrTime_cos

##### Explaination -

1. **Converting 'hhmm' to Minutes:**
   - `df_filtered[column + '_hour'] * 60 + df_filtered[column + '_minute']`: This part converts the 'hhmm' format to minutes. It takes the hour component, multiplies it by 60 to convert to minutes, and adds the minute component.

2. **Normalizing to [0, 1):**
   - `(df_filtered[column + '_hour'] * 60 + df_filtered[column + '_minute']) / (24 * 60)`: This part normalizes the minutes to the range [0, 1) by dividing by the total number of minutes in a day (24 hours * 60 minutes).

3. **Trigonometric Transformations:**
   - `2 * np.pi * ...`: This part scales the normalized time to the range [0, 2π), making it suitable for trigonometric functions.

   - `np.sin(...)`: This calculates the sine of the scaled, normalized time. The sine function produces a wave-like pattern with values between -1 and 1. It captures the cyclical nature of time, where similar times of day will have similar sine values.

   - `np.cos(...)`: Similarly, this calculates the cosine of the scaled, normalized time. The cosine function is another periodic function that is shifted relative to the sine function. It provides an additional perspective on the cyclical pattern.

In summary, the sine and cosine transformations encode the time information into two continuous features that capture both the cyclicality and the phase (shift) within the day. This encoding is often beneficial for machine learning models to capture the temporal patterns in a way that respects the circular nature of time. The resulting features (`column + '_sin'` and `column + '_cos'`) can be used as inputs to your machine learning model for predicting flight cancellations.

### - Converting columns with Boolean values to Binary

In [237]:

df_filtered['Cancelled'] = df_filtered['Cancelled'].astype(int)
df_filtered['Diverted'] = df_filtered['Diverted'].astype(int)



# Print data types of the updated DataFrame
print(df_filtered.dtypes) 

Airline                    object
Origin                     object
Cancelled                   int32
Diverted                    int32
DepDelayMinutes           float64
ArrDelayMinutes           float64
AirTime                   float64
CRSElapsedTime            float64
ActualElapsedTime         float64
Distance                  float64
Month                       int64
DayofMonth                  int64
DayOfWeek                   int64
DepDel15                  float64
DepartureDelayGroups      float64
TaxiOut                   float64
TaxiIn                    float64
ArrDel15                  float64
ArrivalDelayGroups        float64
DistanceGroup               int64
DivAirportLandings          int64
Delay                       int32
CRSDepTime_sin            float64
CRSDepTime_cos            float64
DepTime_sin               float64
DepTime_cos               float64
ArrTime_sin               float64
ArrTime_cos               float64
CRSArrTime_sin            float64
CRSArrTime_cos

### - Handling missing values in the dataset

In [238]:
# Verify that missing values are filled
missing_values_after_imputation = df_filtered.isnull().sum()
print("Missing Values :")
print(missing_values_after_imputation)

Missing Values :
Airline                       0
Origin                        0
Cancelled                     0
Diverted                      0
DepDelayMinutes           22680
ArrDelayMinutes           25261
AirTime                   25261
CRSElapsedTime                0
ActualElapsedTime         25261
Distance                      0
Month                         0
DayofMonth                    0
DayOfWeek                     0
DepDel15                  22680
DepartureDelayGroups      22680
TaxiOut                   23032
TaxiIn                    23436
ArrDel15                  25261
ArrivalDelayGroups        25261
DistanceGroup                 0
DivAirportLandings            0
Delay                         0
CRSDepTime_sin                0
CRSDepTime_cos                0
DepTime_sin               22680
DepTime_cos               22680
ArrTime_sin               23436
ArrTime_cos               23436
CRSArrTime_sin                0
CRSArrTime_cos                0
WheelsOff_sin          

In [181]:
df_filtered[['DepDelayMinutes','AirTime','DepDel15','DepartureDelayGroups','TaxiOut','TaxiIn','DepTime_sin','DepTime_cos','WheelsOff_sin','WheelsOff_cos','WheelsOn_sin','WheelsOn_cos']]

,DepDelayMinutes,AirTime,DepDel15,DepartureDelayGroups,TaxiOut,TaxiIn,DepTime_sin,DepTime_cos,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos
5,0.0,77.0,0.0,-1.0,25.0,4.0,0.529919,-0.848048,0.434445,-0.900698,-0.147809,-0.989016
10,6.0,45.0,0.0,0.0,13.0,6.0,-0.833886,-0.551937,-0.863836,-0.503774,-0.945519,-0.325568
15,0.0,39.0,0.0,-1.0,12.0,3.0,0.584250,-0.811574,0.540974,-0.841039,0.390731,-0.920505
16,0.0,45.0,0.0,-1.0,18.0,2.0,-0.350207,-0.936672,-0.422618,-0.906308,-0.591310,-0.806445
17,0.0,45.0,0.0,-1.0,12.0,4.0,0.207912,-0.978148,0.156434,-0.987688,-0.039260,-0.999229
...,...,...,...,...,...,...,...,...,...,...,...,...
4078284,2.0,97.0,0.0,0.0,37.0,11.0,-0.386711,-0.922201,-0.529919,-0.848048,-0.946930,-0.321439
4078288,69.0,136.0,1.0,4.0,12.0,5.0,0.021815,-0.999762,-0.030539,-0.999534,-0.774393,-0.632705
4078304,0.0,102.0,0.0,-1.0,18.0,6.0,0.008727,-0.999962,-0.069756,-0.997564,-0.492424,-0.870356
4078305,54.0,79.0,1.0,3.0,21.0,5.0,-0.121869,-0.992546,-0.212178,-0.977231,-0.529919,-0.848048


<strong>Notes:</strong>

These columns will be filled by most frequent items.
- 'DepDel15' column has binary column that has 1 if the flight was delaued by 15 mins and 0 if not. We will fill it using most frequent item.
- 'Origin' and 'Airline' represent origin airport and flight airlines taking off those airports. For this we are considering 5 airports and 20 airlines. 

 These columns would be filled with mean of each columns respectively.
- 'DepDelayMinutes' is difference in minutes between scheduled and actual departure time. Early departures set to 0. 
- 'TaxiIn' and 'TaxiOut' are taxi in and taxi out time represented in minutes.
- 'AirTime' is flight duration time, in minutes.
- 'DepartureDelayGroups' Departure Delay intervals, every (15 minutes from 180)
-  [DepTime_sin	DepTime_cos	WheelsOff_sin	WheelsOff_cos	WheelsOn_sin	WheelsOn_cos] this columns were in hhmm format then converted using sine and cosine function to capture cylical nature of time. Thus, they all represent time. 

<strong>Pipeline</strong>


In [182]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define numerical and categorical columns
num_cols = df_filtered.select_dtypes(include='number').columns.to_list()
cat_cols = df_filtered.select_dtypes(exclude='number').columns.to_list()

# Remove the target column from numerical columns


# Create pipelines for numerical and categorical columns
num_pipeline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

# Use ColumnTransformer to set the estimators and transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ],
    remainder='passthrough'
)


In [183]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['DepDelayMinutes', 'AirTime',
                                  'CRSElapsedTime', 'Distance', 'Month',
                                  'DayofMonth', 'DayOfWeek', 'DepDel15',
                                  'DepartureDelayGroups', 'TaxiOut', 'TaxiIn',
                                  'DistanceGroup', 'DivAirportLandings',
                                  'CRSDepTime_sin', 'C...
                                  'CRSArrTime_sin', 'CRSArrTime_cos',
                                  'WheelsOff_sin', 'WheelsOff_cos',
                                  'WheelsOn_sin', 'WheelsOn_cos',
                                  'ArrTimeBlk_numeric_sin',
                                  'ArrTimeBlk_numeric_cos',
                                  'DepTimeBlk_numeric_sin',
                                  'DepTimeBlk_numeric_cos',
                                  'Cancelled_binary']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Airline', 'Origin'])])

In [184]:
# Apply the preprocessing pipeline on the dataset
df_prepared = preprocessor.fit_transform(df_filtered)

# Scikit-learn strips the column headers in most cases, so just add them back on afterward.
feature_names = preprocessor.get_feature_names_out()

df_prepared = pd.DataFrame(data=df_prepared, columns=feature_names)

df_prepared.head()

,num__DepDelayMinutes,num__AirTime,num__CRSElapsedTime,num__Distance,num__Month,num__DayofMonth,num__DayOfWeek,num__DepDel15,num__DepartureDelayGroups,num__TaxiOut,...,cat__Airline_SkyWest Airlines Inc.,cat__Airline_Southwest Airlines Co.,cat__Airline_Spirit Air Lines,cat__Airline_United Air Lines Inc.,cat__Origin_ATL,cat__Origin_CLT,cat__Origin_DEN,cat__Origin_DFW,cat__Origin_ORD,remainder__Diverted_binary
0,-0.333564,-0.416782,-0.439979,-0.384528,-0.041377,-1.336229,-1.503005,-0.55114,-0.549154,0.719060,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.206645,-0.968354,-1.022715,-1.029048,-0.041377,-1.336229,-1.503005,-0.55114,-0.105606,-0.593989,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.333564,-1.071774,-1.056014,-1.066351,-0.041377,-1.336229,-1.503005,-0.55114,-0.549154,-0.703409,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.333564,-0.968354,-0.956117,-0.985527,-0.041377,-1.336229,-1.503005,-0.55114,-0.549154,-0.046885,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.333564,-0.968354,-0.872869,-0.877762,-0.041377,-1.336229,-1.503005,-0.55114,-0.549154,-0.703409,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [185]:
missing_values_count = df_prepared.isnull().sum()
print("Missing Values Count for Each Column:")
print(missing_values_count)


Missing Values Count for Each Column:
num__DepDelayMinutes                                      0
num__AirTime                                              0
num__CRSElapsedTime                                       0
num__Distance                                             0
num__Month                                                0
num__DayofMonth                                           0
num__DayOfWeek                                            0
num__DepDel15                                             0
num__DepartureDelayGroups                                 0
num__TaxiOut                                              0
num__TaxiIn                                               0
num__DistanceGroup                                        0
num__DivAirportLandings                                   0
num__CRSDepTime_sin                                       0
num__CRSDepTime_cos                                       0
num__DepTime_sin                                          0
nu

In [186]:
# data types of the columns
data_types = df_prepared.dtypes
print("Data Types of Columns:")
print(data_types)


Data Types of Columns:
num__DepDelayMinutes                                      float64
num__AirTime                                              float64
num__CRSElapsedTime                                       float64
num__Distance                                             float64
num__Month                                                float64
num__DayofMonth                                           float64
num__DayOfWeek                                            float64
num__DepDel15                                             float64
num__DepartureDelayGroups                                 float64
num__TaxiOut                                              float64
num__TaxiIn                                               float64
num__DistanceGroup                                        float64
num__DivAirportLandings                                   float64
num__CRSDepTime_sin                                       float64
num__CRSDepTime_cos                                  

<strong>Notes:</strong>

- Columns prefixed with num__ are numerical features.
- Columns prefixed with cat__ are one-hot encoded categorical features.
- The column remainder__Diverted_binary is the binary feature for Diverted_binary.

All columns are of type float64, which might be due to the encoding performed during preprocessing. However, the columns representing binary features (Cancelled_binary and Diverted_binary) should ideally be of integer type (0 or 1) instead of float. We may need to convert these columns to integer type if they are supposed to represent binary values. Additionally, it seems that Cancelled_binary is missing from the output. We need to ensure that it's properly included in the preprocessing pipeline.

In [187]:
# Converting target column to Int
# Convert binary columns to integer type
df_prepared['remainder__Diverted_binary'] = df_prepared['remainder__Diverted_binary'].astype(int)
df_prepared['remainder__Diverted_binary'].dtypes


dtype('int32')

In [188]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define X (features) and y (target)
X = df_prepared.drop(columns=['remainder__Diverted_binary'])
y = df_prepared['remainder__Diverted_binary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Support Vector Machine (SVM) model
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM Accuracy:", svm_accuracy)

# Train Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)

# Train Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_accuracy)


SVM Accuracy: 0.99999375986721
Random Forest Accuracy: 1.0
Logistic Regression Accuracy: 0.9999750394688399


,num__DepDelayMinutes,num__AirTime,num__CRSElapsedTime,num__Distance,num__Month,num__DayofMonth,num__DayOfWeek,num__DepDel15,num__DepartureDelayGroups,num__TaxiOut,...,cat__Airline_Republic Airlines,cat__Airline_SkyWest Airlines Inc.,cat__Airline_Southwest Airlines Co.,cat__Airline_Spirit Air Lines,cat__Airline_United Air Lines Inc.,cat__Origin_ATL,cat__Origin_CLT,cat__Origin_DEN,cat__Origin_DFW,cat__Origin_ORD
183922,-0.333564,3.137102e-02,0.176055,0.005086,-1.038177,1.404605,-1.503005,-5.511402e-01,-5.491541e-01,6.096390e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
724754,-0.333564,-1.582320e-01,-0.023740,0.013375,-0.539777,-0.651021,-0.007909,-5.511402e-01,-5.491541e-01,2.579212e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
269460,-0.333564,7.380731e-01,0.875338,0.979120,1.453823,-0.194215,-0.007909,-5.511402e-01,-5.491541e-01,-3.751473e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
241450,-0.333564,1.220699e+00,1.224979,1.337647,1.453823,-0.993625,-0.007909,-5.511402e-01,-5.491541e-01,-8.128300e-01,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
77880,0.237573,5.484701e-01,0.525697,0.813327,-0.041377,-1.222028,-1.004640,1.867274e+00,3.379420e-01,1.719562e-01,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,-0.333564,-1.020064e+00,-0.972766,-0.968948,1.453823,1.176202,-1.004640,-5.511402e-01,-5.491541e-01,-8.128300e-01,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
365838,-0.333564,-1.071774e+00,-1.022715,-1.045627,0.457023,1.633007,-1.503005,-5.511402e-01,-5.491541e-01,-2.657266e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
131932,-0.333564,-4.857281e-01,-0.356731,-0.469497,-1.038177,1.290403,1.487187,-5.511402e-01,-5.491541e-01,-7.034093e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
671155,0.000000,-7.348420e-16,-0.989416,-1.041482,-1.536577,-0.422618,-0.506274,2.013734e-16,-2.462186e-17,-3.887404e-16,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
